## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [12]:
import json
import numpy as np
from openai import AsyncOpenAI
import pandas as pd
from pydantic import BaseModel
from tqdm.auto import tqdm

from lattereview.providers import OpenAIProvider
from lattereview.providers import LiteLLMProvider
from lattereview.agents import AbstractionReviewer
from lattereview.workflows import ReviewWorkflow

## Data

Building five example stories and dummy question-answering pairs from each story:

In [6]:
class BuildStoryOutput(BaseModel):
    story: str
    location: str
    characters: list[str]

async def build_story():
    prompt = """
    Write a one-paragraph story with whatever realistic or imaginary theme you like,  
    then create a list of all characters you named in your story.
    Return your story, the main location that your story happens in, and a Python list of your characters as your output.
    """
    provider = OpenAIProvider(model="gpt-4o", response_format_class=BuildStoryOutput)
    return await provider.get_json_response(prompt, temperature=0.9)

def run_build_story():
    response =  asyncio.run(build_story())[0]
    return response

data = {
    "story": [],
    "location": [],
    "characters": [],
}
for i in tqdm(range(5)):
    out = json.loads(run_build_story())
    data["characters"].append(out["characters"])
    data["location"].append(out["location"])
    data["story"].append(out["story"])


data = pd.DataFrame(data)
data.to_csv("data.csv", index=False)
data

100%|██████████| 5/5 [00:20<00:00,  4.06s/it]


,story,location,characters
0,"Deep within the enchanted forest of Ellowyn, w...",Enchanted forest of Ellowyn,"[Elara, Mirin, Arion]"
1,In the heart of the enchanted forest of Alvero...,Enchanted Forest of Alveron,"[Elara, Finn]"
2,"In the tiny coastal village of Marlin Bay, nes...",Marlin Bay,"[Amelia, Oliver]"
3,"In the heart of the Enchanted Forest, where th...",Enchanted Forest,"[Orin, Felix]"
4,In the heart of the enchanted forest of Elderg...,Elderglen forest,"[Alara, Seraphina]"


## Abstraction with a single agent

In [31]:
Albert = AbstractionReviewer(
    provider=LiteLLMProvider(model="gpt-4o-mini"),
    name="Albert",
    max_concurrent_requests=1, 
    backstory="an expert reviewer!",
    input_description = "stories",
    model_args={"max_tokens": 200, "temperature": 0.1},
    response_format = {
        "location": str, 
        "characters": list[str]
    },
    instructions = {
        "location": "The main location that the story happens in.", 
        "characters": "The name of the characters mentioned in the story."
    }
)


# Dummy input
input_list = data.story.str.lower().tolist()
print("====== Inputs ======\n\n", '\n'.join(input_list))

# Dummy review
results, total_cost = asyncio.run(Albert.review_items(input_list))
print("\n====== Outputs ======")
for result in results:
    print(result)

# Dummy costs
print("\n====== Costs ======\n")
for i, item in enumerate(Albert.memory):
    print(f"Cost for item {i}: {item['cost']}")

print(f"\nTotal cost: {total_cost}")

====== Inputs ======

 deep within the enchanted forest of ellowyn, where the trees whispered ancient secrets and the brook sang to the moon, young elara stumbled upon a hidden glade bathed in golden light. the air was thick with the scent of blooming nightshade, and the soft trills of creatures unseen mingled with the rustle of leaves. as elara explored, she met mirin, a wise old sprite with eyes like twin emeralds and wings that shimmered like a thousand rainbows. together, they uncovered the forgotten ruins of an ancient empire, long lost to time, and awakened the gentle spirit of arion, the last guardian of the forest. with the first light of dawn, elara and her newfound friends vowed to protect the magic of ellowyn from the encroaching shadows of the outside world.
in the heart of the enchanted forest of alveron, the wise old owl named elara watched over the land from her perch atop the ancient oak tree. she had seen the forest come to life with the magic of spring, witnessed the 

Reviewing 5 items - 2024-12-27 22:34:40: 100%|██████████| 5/5 [00:07<00:00,  1.49s/it]


====== Outputs ======
{'location': 'the enchanted forest of ellowyn', 'characters': ['elara', 'mirin', 'arion']}
{'location': 'the enchanted forest of alveron', 'characters': ['elara', 'finn']}
{'location': 'the tiny coastal village of Marlin Bay', 'characters': ['Amelia', 'Oliver']}
{'location': 'the heart of the enchanted forest', 'characters': ['orin', 'felix']}
{'location': 'the enchanted forest of elderglen', 'characters': ['alara', 'seraphina']}

====== Costs ======

Cost for item 0: 7.14e-05
Cost for item 1: 7.14e-05
Cost for item 2: 6.42e-05
Cost for item 3: 7.26e-05
Cost for item 4: 6.869999999999999e-05

Total cost: 6.869999999999999e-05


In [32]:
Albert.identity

{'system_prompt': "Your name is: <<Albert>> Your backstory is: <<an expert reviewer!>>. Your task is to review input itmes with the following description: <<stories>>. Your final output should have the following keys: location (<class 'str'>), characters (list[str]).",
 'formatted_prompt': "**Review the input item below and extract the specified keys as instructed:** --- **Input Item:** <<${item}$>> **Keys to Extract and Their Expected Formats:** <<{'location': <class 'str'>, 'characters': list[str]}>> --- **Instructions:** Follow the detailed guidelines below for extracting the specified keys: <<{'location': 'The main location that the story happens in.', 'characters': 'The name of the characters mentioned in the story.'}>> --- <<${additional_context}$>>",
 'model_args': {'max_tokens': 200, 'temperature': 0.1}}

## Abstraction with a workflow

In [33]:
workflow = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [Albert],
            "text_inputs": ["story"]
        }
    ]
)

# Reload the data if needed.
updated_data = asyncio.run(workflow(data))

print("\n====== Costs ======\n")
print("Total cost: ", workflow.get_total_cost())
print("Detailed costs: ", workflow.reviewer_costs)

updated_data


====== Starting review round A (1/1) ======

Processing 5 eligible rows


['round: A', 'reviewer_name: Albert'] -                     2024-12-27 22:35:37: 100%|██████████| 5/5 [00:06<00:00,  1.25s/it]

The following columns are present in the dataframe at the end of Albert's reivew in round A: ['story', 'location', 'characters', 'round-A_Albert_output', 'round-A_Albert_location', 'round-A_Albert_characters']

====== Costs ======

Total cost:  7.08e-05
Detailed costs:  {('A', 'Albert'): 7.08e-05}


,story,location,characters,round-A_Albert_output,round-A_Albert_location,round-A_Albert_characters
0,"Deep within the enchanted forest of Ellowyn, w...",Enchanted forest of Ellowyn,"[Elara, Mirin, Arion]",{'location': 'the enchanted forest of Ellowyn'...,the enchanted forest of Ellowyn,"[Elara, Mirin, Arion]"
1,In the heart of the enchanted forest of Alvero...,Enchanted Forest of Alveron,"[Elara, Finn]",{'location': 'the enchanted forest of Alveron'...,the enchanted forest of Alveron,"[Elara, Finn]"
2,"In the tiny coastal village of Marlin Bay, nes...",Marlin Bay,"[Amelia, Oliver]","{'location': 'Marlin Bay', 'characters': ['Ame...",Marlin Bay,"[Amelia, Oliver]"
3,"In the heart of the Enchanted Forest, where th...",Enchanted Forest,"[Orin, Felix]","{'location': 'Enchanted Forest', 'characters':...",Enchanted Forest,"[Orin, Felix]"
4,In the heart of the enchanted forest of Elderg...,Elderglen forest,"[Alara, Seraphina]",{'location': 'the enchanted forest of Eldergle...,the enchanted forest of Elderglen,"[Alara, Seraphina]"
